# PyVista - NASA P3 Case Directory Read & Analysis

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pyvista as pv

## Define Helper Functions to Slice the Internal Mesh

In [2]:
def slice_center(mesh, 
                 NORMAL='z', 
                 ORIGIN=[0, 0, 0], 
                 translate=False, 
                 CONTOUR=False
                ):
    """Slice mesh through center in normal direction, move to zero normal."""
    slice_mesh = mesh.slice(normal=NORMAL, origin=ORIGIN, contour=CONTOUR)
    if translate is not False:
        if NORMAL == 'z':
            slice_mesh.translate((0, 0, -slice_mesh.center[-1]), inplace=True)
        elif NORMAL == 'x':
            slice_mesh.translate((-slice.mesh.center[0], 0, 0), inplace=True)
        elif NORMAL == 'y':
            slice_mesh.translate((0, -slice.mesh.center[1], 0), inplace=True)
        else:
            print('ERROR: Normal not found - ', NORMAL)
            slice_mesh = None
    return slice_mesh

In [3]:
def slice_multiple(mesh, NORMAL='z', nslices=2, CONTOUR=False):
    """Slice mesh multiple times along specified axis, move to z=0."""
    slice_mesh = mesh.slice_along_axis(n=nslices, axis=NORMAL, contour=CONTOUR)
    #slice_mesh.translate((0, 0, -slice_mesh.center[-1]), inplace=True)
    return slice_mesh

## Define the Case Directory

In [4]:
case_directory = '/Users/jrobrien/Dissertation/data/solutions/NASA_noPylons_v2_tas120_aoa0_900T33/'

In [5]:
reader = pv.POpenFOAMReader(case_directory + 'NASA_noPylons_v2_tas120_aoa0_900T33.OpenFOAM')

### Subset the final run 

In [6]:
reader.time_values[-1]

786.0

In [7]:
%%time
reader.set_active_time_value(reader.time_values[-1])
reader.cell_to_point_creation = True
mesh = reader.read()

CPU times: user 4.64 s, sys: 698 ms, total: 5.34 s
Wall time: 5.31 s


### Define the mesh portions

In [8]:
internal_mesh = mesh["internalMesh"]
boundaries = mesh["boundary"]

In [9]:
# inlet velocity
boundaries['inlet']['U'][0]

pyvista_ndarray([120.,   0.,   0.], dtype=float32)

In [10]:
internal_mesh

UnstructuredGrid (0x15443fb80)
  N Cells:    10691736
  N Points:   11526824
  X Bounds:   -2.000e+02, 2.000e+02
  Y Bounds:   -2.000e+02, 2.000e+02
  Z Bounds:   -2.000e+02, 2.000e+02
  N Arrays:   18

## Manipulate the Scalars - Cell Values

In [11]:
## scale velocities based on input patch velocities
## NOTE: only works on cells!
internal_mesh['U'] = internal_mesh['U'] / boundaries['inlet']['U'][0][0]

In [12]:
internal_mesh

UnstructuredGrid (0x15443fb80)
  N Cells:    10691736
  N Points:   11526824
  X Bounds:   -2.000e+02, 2.000e+02
  Y Bounds:   -2.000e+02, 2.000e+02
  Z Bounds:   -2.000e+02, 2.000e+02
  N Arrays:   18

## Slice Mesh to View Aircraft - Plot

### Z-slice - move origin

In [19]:
slice_meshA = slice_center(internal_mesh, NORMAL='z', CONTOUR=False)
pt = pv.Plotter()
# Note: clim setting allows for 1.0 to be neutral gray
pt.add_mesh(slice_meshA, scalars='U', preference='cell', cmap='bwr', clim=[0.55, 1.45])
pt.show(cpos="xy")

2023-05-24 22:43:39.222 ( 134.673s) [           297D6]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.
2023-05-24 22:43:39.308 ( 134.759s) [           297D6]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.


Widget(value="<iframe src='http://localhost:55214/index.html?ui=P_0x2dff7cc70_5&reconnect=auto' style='width: …

In [ ]:
sliceA_contour = slice_center(internal_mesh, NORMAL='z', CONTOUR=False)
pt = pv.Plotter()
# Note: clim setting allows for 1.0 to be neutral gray
pt.add_mesh(slice_meshA, scalars='U', preference='cell', cmap='bwr', clim=[0.55, 1.45])
pt.show(cpos="xy")

## Z-slice - with displaying the Point Values

In [20]:
slice_meshB = slice_center(internal_mesh, NORMAL='z', CONTOUR=False)
pt = pv.Plotter()
# Note: clim setting allows for 1.0 to be neutral gray
pt.add_mesh(slice_meshB, scalars='U', preference='point', cmap='bwr', clim=[85, 155])
pt.show(cpos="xy")

2023-05-24 22:43:55.501 ( 150.952s) [           297D6]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.
2023-05-24 22:43:55.587 ( 151.038s) [           297D6]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.


Widget(value="<iframe src='http://localhost:55214/index.html?ui=P_0x3ce5e4b80_6&reconnect=auto' style='width: …

### y-axis slice - with change in origin

In [21]:
slice3 = slice_center(mesh['internalMesh'], NORMAL='y')

In [22]:
# Note you can change the origin of the slice
slice3 = slice_center(mesh['internalMesh'], NORMAL='y', ORIGIN=[0, 0, 5])
pt = pv.Plotter()
pt.add_mesh(slice3, scalars='U')
pt.show(cpos="zx")

Widget(value="<iframe src='http://localhost:55214/index.html?ui=P_0x3c68f5b80_7&reconnect=auto' style='width: …

### x-slice - with and without contours

In [17]:
slice4 = slice_center(mesh['internalMesh'], NORMAL='x', ORIGIN=[15, 0, 0], CONTOUR=False)
pt = pv.Plotter()
pt.add_mesh(slice4, scalars='U', cmap='bwr', clim=[85, 155])
pt.show(cpos="zy")

Widget(value="<iframe src='http://localhost:55214/index.html?ui=P_0x3c68c7d60_3&reconnect=auto' style='width: …

In [23]:
slice4B = slice_center(mesh['internalMesh'], NORMAL='x', ORIGIN=[15, 0, 0], CONTOUR=True)
pt = pv.Plotter()
pt.add_mesh(slice4, scalars='U', cmap='bwr', clim=[85, 155])
pt.show(cpos="zy")

Widget(value="<iframe src='http://localhost:55214/index.html?ui=P_0x3c68ede20_8&reconnect=auto' style='width: …

### Polyslice (not really helpful here)

In [24]:
sliceA = slice_multiple(mesh['internalMesh'], NORMAL='z', nslices=10, CONTOUR=False)
sliceB = slice_multiple(mesh['internalMesh'], NORMAL='x', nslices=10, CONTOUR=False)
sliceB = slice_multiple(mesh['internalMesh'], NORMAL='y', nslices=10, CONTOUR=False)
pt = pv.Plotter()
pt.add_mesh(sliceA, scalars='U')
pt.show(cpos="xy")

Widget(value="<iframe src='http://localhost:55214/index.html?ui=P_0x3c68f8820_9&reconnect=auto' style='width: …

## Contrasting Solutions

In [ ]:
# Define a Pylon Solution
navy_directory = '/Users/jrobrien/Dissertation/data/solutions/NASA_extendedPylon_v2_tas120_aoa0_900T33/'
navy = pv.POpenFOAMReader(navy_directory + 'NASA_extendedPylon_v2_tas120_aoa0_900T33.OpenFOAM')

In [ ]:
navy.time_values[-1]

In [ ]:
%%time
navy.set_active_time_value(navy.time_values[-1])
navy.cell_to_point_creation = True
navy_mesh = navy.read()

In [ ]:
navy_internal = navy_mesh["internalMesh"]
navy_boundaries = navy_mesh["boundary"]